# Lab 2.5.6: Uploading Models to the Hugging Face Hub

**Module:** 2.5 - Hugging Face Ecosystem  
**Time:** 2 hours  
**Difficulty:** ⭐⭐ (Intermediate)

---

## Learning Objectives

By the end of this lab, you will:
- [ ] Authenticate with the Hugging Face Hub
- [ ] Create a comprehensive model card
- [ ] Push a fine-tuned model to the Hub
- [ ] Use your uploaded model with `pipeline()`

---

## Prerequisites

- Completed: Labs 2.5.1 through 2.5.5
- **HF Account**: Create one at https://huggingface.co/join
- **Access Token**: Get one from https://huggingface.co/settings/tokens

---

## Real-World Context

**Sharing is Caring**: You've just fine-tuned an amazing sentiment classifier. Now you can:
- Share it with your team (private models)
- Share it with the world (public models)
- Use it in any project with one line: `pipeline("your-username/your-model")`

The Hub is not just storage - it's your AI portfolio, demonstrating your skills to employers and collaborators!

---

## ELI5: Why Share Models?

> **Imagine you baked an amazing cake...**
>
> You could keep the recipe secret, or you could share it so others can:
> - Make the same delicious cake
> - Improve upon your recipe
> - Give you credit for the original
>
> The Hugging Face Hub is like a cookbook where everyone can:
> - Download your "recipe" (model weights)
> - See the "instructions" (model card)
> - Try before they download (hosted inference)
> - Build upon your work (fork and modify)

---

## Part 1: Authentication

In [ ]:
import torch
from huggingface_hub import HfApi, login, create_repo, upload_folder
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import os

print("Environment Check")
print("=" * 50)
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")

In [ ]:
# Login to Hugging Face Hub
# Option 1: Interactive login (recommended)
login()

# Option 2: Use environment variable
# os.environ['HF_TOKEN'] = 'your_token_here'

# Option 3: Pass token directly (not recommended for shared notebooks)
# login(token='your_token_here')

In [ ]:
# Verify authentication
api = HfApi()
user_info = api.whoami()

print(f"\nLogged in as: {user_info['name']}")
print(f"Username: {user_info['fullname'] if 'fullname' in user_info else 'N/A'}")

USERNAME = user_info['name']
print(f"\nYour models will be at: huggingface.co/{USERNAME}/...")

---

## Part 2: Preparing a Model for Upload

In [ ]:
from datasets import load_dataset
from transformers import Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model, TaskType
import evaluate
import numpy as np

# Let's fine-tune a quick sentiment model to upload
print("Step 1: Load base model and dataset")
print("-" * 50)

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    torch_dtype=torch.bfloat16
)

# Load small subset for quick training
dataset = load_dataset("imdb")
train_data = dataset['train'].shuffle(seed=42).select(range(2000))
eval_data = dataset['test'].shuffle(seed=42).select(range(500))

print(f"Base model: {model_name}")
print(f"Train examples: {len(train_data)}")
print(f"Eval examples: {len(eval_data)}")

In [ ]:
# Apply LoRA for efficient fine-tuning
print("\nStep 2: Apply LoRA")
print("-" * 50)

lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["q_lin", "v_lin"],
    modules_to_save=["classifier", "pre_classifier"]
)

model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

In [ ]:
# Tokenize data
print("\nStep 3: Tokenize")
print("-" * 50)

def tokenize_fn(examples):
    return tokenizer(
        examples['text'],
        padding='max_length',
        truncation=True,
        max_length=256
    )

tokenized_train = train_data.map(tokenize_fn, batched=True, remove_columns=['text'])
tokenized_eval = eval_data.map(tokenize_fn, batched=True, remove_columns=['text'])

tokenized_train = tokenized_train.rename_column("label", "labels")
tokenized_eval = tokenized_eval.rename_column("label", "labels")

print("Tokenization complete!")

In [ ]:
# Train
print("\nStep 4: Train")
print("-" * 50)

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(
    output_dir="./model_for_upload",
    num_train_epochs=2,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    learning_rate=3e-4,
    warmup_ratio=0.1,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    bf16=True,
    logging_steps=50,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()
print("Training complete!")

In [ ]:
# Evaluate final model
print("\nStep 5: Final Evaluation")
print("-" * 50)

eval_results = trainer.evaluate()
print(f"Accuracy: {eval_results['eval_accuracy']:.4f}")

In [ ]:
# Merge LoRA weights and save
print("\nStep 6: Merge LoRA and Save")
print("-" * 50)

merged_model = model.merge_and_unload()

save_path = "./model_for_upload/final"
merged_model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"Model saved to {save_path}")

# Check files
print("\nSaved files:")
for f in os.listdir(save_path):
    size = os.path.getsize(os.path.join(save_path, f)) / 1e6
    if size > 0.1:
        print(f"  {f}: {size:.1f} MB")

---

## Part 3: Creating a Model Card

The **Model Card** is crucial - it tells users everything they need to know about your model.

In [ ]:
# Create a comprehensive model card
model_card_content = f"""
---
language:
- en
license: apache-2.0
tags:
- sentiment-analysis
- text-classification
- distilbert
- lora
- imdb
datasets:
- imdb
metrics:
- accuracy
base_model: distilbert-base-uncased
model-index:
- name: imdb-sentiment-classifier
  results:
  - task:
      type: text-classification
      name: Sentiment Analysis
    dataset:
      type: imdb
      name: IMDB
    metrics:
    - type: accuracy
      value: {eval_results['eval_accuracy']:.4f}
---

# IMDB Sentiment Classifier

This model is a fine-tuned version of [distilbert-base-uncased](https://huggingface.co/distilbert-base-uncased) on the IMDB movie review dataset for sentiment analysis.

## Model Description

- **Base Model:** distilbert-base-uncased
- **Task:** Binary sentiment classification (positive/negative)
- **Dataset:** IMDB movie reviews
- **Fine-tuning Method:** LoRA (Low-Rank Adaptation)
- **Training Framework:** Hugging Face Transformers + PEFT

## Intended Use

This model is designed for sentiment analysis of English text, particularly movie reviews and similar entertainment-related content.

### Direct Use

```python
from transformers import pipeline

classifier = pipeline("text-classification", model="{USERNAME}/imdb-sentiment-classifier")
result = classifier("This movie was absolutely fantastic!")
print(result)
# [{{{'label': 'LABEL_1', 'score': 0.98}}}}]  # LABEL_1 = Positive
```

### Labels

- **LABEL_0:** Negative sentiment
- **LABEL_1:** Positive sentiment

## Training Details

### Training Data

The model was fine-tuned on a subset of the IMDB dataset:
- Training examples: 2,000
- Evaluation examples: 500

### Training Hyperparameters

- **Epochs:** 2
- **Batch size:** 32
- **Learning rate:** 3e-4
- **LoRA rank:** 8
- **LoRA alpha:** 16
- **Precision:** bfloat16

### Training Results

| Metric | Value |
|--------|-------|
| Accuracy | {eval_results['eval_accuracy']:.4f} |
| Eval Loss | {eval_results['eval_loss']:.4f} |

## Limitations

- Trained primarily on movie reviews; may not generalize well to other domains
- English only
- May reflect biases present in the training data

## Environmental Impact

- **Hardware:** NVIDIA DGX Spark (128GB unified memory)
- **Training time:** ~5 minutes
- **Carbon emission:** Minimal (local desktop training)

## Citation

```bibtex
@misc{{imdb-sentiment-classifier,
  author = {{{USERNAME}}},
  title = {{IMDB Sentiment Classifier}},
  year = {{2025}},
  publisher = {{Hugging Face}},
  journal = {{Hugging Face Hub}},
}}
```
"""

# Save the model card
with open(os.path.join(save_path, "README.md"), "w") as f:
    f.write(model_card_content)

print("Model card created!")
print("\nPreview:")
print("-" * 60)
print(model_card_content[:1000] + "...")

---

## Part 4: Uploading to the Hub

In [ ]:
# Define repository name
REPO_NAME = "imdb-sentiment-classifier"
REPO_ID = f"{USERNAME}/{REPO_NAME}"

print(f"Repository ID: {REPO_ID}")
print(f"URL: https://huggingface.co/{REPO_ID}")

In [ ]:
# Create repository on Hub
print("\nCreating repository...")

try:
    repo_url = create_repo(
        repo_id=REPO_NAME,
        repo_type="model",
        private=False,  # Set to True for private model
        exist_ok=True   # Don't error if it already exists
    )
    print(f"Repository created/exists: {repo_url}")
except Exception as e:
    print(f"Note: {e}")

In [ ]:
# Upload model files
print("\nUploading model...")

api.upload_folder(
    folder_path=save_path,
    repo_id=REPO_ID,
    repo_type="model",
    commit_message="Upload fine-tuned IMDB sentiment classifier"
)

print(f"\nModel uploaded to: https://huggingface.co/{REPO_ID}")

### Alternative: Use push_to_hub()

In [ ]:
# Alternative method: push_to_hub() from the model itself
# This is often simpler:

# merged_model.push_to_hub(REPO_ID, commit_message="Initial upload")
# tokenizer.push_to_hub(REPO_ID, commit_message="Add tokenizer")

# Or use the trainer:
# trainer.push_to_hub(commit_message="Training complete")

---

## Part 5: Using Your Uploaded Model

In [ ]:
# Now anyone can use your model!
from transformers import pipeline

print(f"Loading model from Hub: {REPO_ID}")
print("-" * 60)

# Load from Hub (this downloads and caches the model)
classifier = pipeline(
    "text-classification",
    model=REPO_ID,
    device=0 if torch.cuda.is_available() else -1
)

print("Model loaded successfully!")

In [ ]:
# Test the loaded model
test_texts = [
    "This movie was an absolute masterpiece! The acting was superb.",
    "Worst film I've seen in years. A complete waste of time.",
    "It was okay. Not great, not terrible.",
    "I can't believe how good this was! Must watch for everyone!",
    "Boring, predictable, and poorly written."
]

print("\nTesting your uploaded model:")
print("=" * 60)

for text in test_texts:
    result = classifier(text)[0]
    label = "POSITIVE" if result['label'] == 'LABEL_1' else "NEGATIVE"
    score = result['score']
    
    emoji = "" if label == "POSITIVE" else ""
    print(f"{emoji} {label:8} ({score:.1%}): {text[:45]}...")

---

## Part 6: Managing Your Model on the Hub

In [ ]:
# Get model info from Hub
print("\nModel Info from Hub:")
print("-" * 60)

model_info = api.model_info(REPO_ID)
print(f"Model ID: {model_info.id}")
print(f"Author: {model_info.author}")
print(f"Downloads: {model_info.downloads}")
print(f"Likes: {model_info.likes}")
print(f"Tags: {model_info.tags}")

In [ ]:
# List your models on the Hub
print("\nYour models on the Hub:")
print("-" * 60)

my_models = api.list_models(author=USERNAME)
for m in my_models:
    print(f"  - {m.id}: {m.downloads} downloads")

In [ ]:
# Update model card (add more info, fix typos, etc.)
# You can do this directly on the Hub website or via API:

# Update specific file
# api.upload_file(
#     path_or_fileobj="./updated_README.md",
#     path_in_repo="README.md",
#     repo_id=REPO_ID,
#     commit_message="Update model card"
# )

---

## Part 7: Best Practices for Model Sharing

In [ ]:
print("MODEL SHARING BEST PRACTICES")
print("=" * 60)

best_practices = """
MODEL CARD:
  Include all YAML metadata (language, license, tags, datasets)
  Describe intended use and limitations clearly
  Add example code that works out of the box
  Document training hyperparameters
  Include performance metrics with evaluation details
  Note environmental impact if significant

LICENSING:
  Choose an appropriate license (apache-2.0, mit, etc.)
  Check base model license compatibility
  Consider commercial use implications

VERSIONING:
  Use meaningful commit messages
  Tag important versions
  Document changes between versions

NAMING:
  Use descriptive names: "task-dataset-model" format
  Examples: "imdb-sentiment-distilbert", "ner-conll03-bert"
  Avoid generic names like "my-model" or "test"

TESTING:
  Always test loading from Hub before announcing
  Include example inputs/outputs
  Verify pipeline() works correctly

PRIVATE MODELS:
  Use private=True for proprietary models
  Share with specific users via organization access
  Consider gated models for sensitive content
"""
print(best_practices)

---

## Try It Yourself: Upload a Different Model

Upload a model trained on a different task or dataset:

In [ ]:
# YOUR CODE HERE
# Ideas:
# 1. Fine-tune on AG News (4-class classification)
# 2. Fine-tune on a NER dataset
# 3. Train a different base model (BERT, RoBERTa)
#
# Steps:
# 1. Train your model
# 2. Create a comprehensive model card
# 3. Upload to Hub
# 4. Test loading from Hub
# 5. Share the link!

# Your code here:
# ...

---

## Common Mistakes

### Mistake 1: Missing Model Card

```python
# Wrong: Upload without README.md
model.save_pretrained(path)
api.upload_folder(path, repo_id)
# Result: No description, no usage examples!

# Right: Always include model card
with open(f"{path}/README.md", "w") as f:
    f.write(model_card_content)
api.upload_folder(path, repo_id)
```

### Mistake 2: Forgetting the Tokenizer

```python
# Wrong: Only save model
model.save_pretrained(path)

# Right: Save both model and tokenizer
model.save_pretrained(path)
tokenizer.save_pretrained(path)  # Don't forget!
```

### Mistake 3: Wrong Pipeline Task

```python
# Wrong: Users try wrong pipeline
classifier = pipeline("text-generation", model=repo_id)  # Error!

# Right: Document correct task in model card
# In README.md:
# ```python
# classifier = pipeline("text-classification", model="user/model")
# ```
```

---

## Checkpoint

You've learned:
- How to authenticate with the Hugging Face Hub
- How to create a comprehensive model card
- How to upload models to the Hub
- How to use uploaded models with pipeline()
- Best practices for model sharing

---

## Congratulations!

You've completed the entire Module 2.5: Hugging Face Ecosystem! You now know how to:
- Explore and use models from the Hub
- Build quick prototypes with pipelines
- Process datasets efficiently
- Fine-tune models with the Trainer API
- Use LoRA for parameter-efficient fine-tuning
- Share your models with the world!

---

## Further Reading

- [Hub Documentation](https://huggingface.co/docs/hub)
- [Model Cards Guide](https://huggingface.co/docs/hub/model-cards)
- [Creating Model Repos](https://huggingface.co/docs/hub/repositories-getting-started)

---

## Cleanup

In [ ]:
# Cleanup
import gc

del model, merged_model, trainer, classifier
gc.collect()
torch.cuda.empty_cache()

print(f"GPU memory after cleanup: {torch.cuda.memory_allocated()/1e9:.2f} GB")
print(f"\nYour model is live at: https://huggingface.co/{REPO_ID}")
print("\nModule 2.5 complete!")